# Fine-tuning Mistral-7B con QLoRA

In [ ]:
!pip install transformers datasets peft bitsandbytes accelerate

In [ ]:
!git clone https://github.com/xaviguardia/llm01.git && cd bank-qa-finetune-hf && python train.py

In [ ]:
!python train.py